In [17]:
%load_ext autoreload
%autoreload 2

import polars as pl

In [3]:
df = pl.read_csv("MDS_data_preprocessed/mrmr_selected/disease/mrna/mrna_fold_0.csv")

In [6]:
annots = pl.read_csv("MDS_data_preprocessed/annotations.csv")

SAMPLE_NAME,1 disease,2 risk,3 mutations (SF3B1only_wt),sample_ids
str,i64,i64,i64,str
"""V1884_S17""",2,2,0,"""V1884"""
"""N58_S18""",1,0,0,"""N58"""
"""V630_S11""",2,1,1,"""V630"""
"""N60_S15""",1,0,0,"""N60"""
"""V1297_S10""",2,2,0,"""V1297"""
…,…,…,…,…
"""V1048_S12""",2,1,0,"""V1048"""
"""V806_S1""",2,2,2,"""V806"""
"""V513_S16""",2,2,1,"""V513"""


In [19]:
def process_mds_annotations(df, column_name):
    mds_y = annots.select(pl.col(column_name), pl.col("sample_ids"))
    # drop all rows where the target is 0
    mds_y = mds_y.filter(pl.col(column_name) != 0)
    # subtract one from the 1 target column
    mds_y = mds_y.with_columns(pl.col(column_name) - 1)

    return mds_y

process_mds_annotations(df, "1 disease").write_csv("disease_annoations.csv")
process_mds_annotations(df, "2 risk").write_csv("risk_annotations.csv")
process_mds_annotations(df, "3 mutations (SF3B1only_wt)").write_csv("mutation_annotations.csv")

In [20]:
disase_annotations = pl.read_csv("disease_annoations.csv")

mrna = pl.read_csv("MDS_data_preprocessed/mrna.csv")
mirna = pl.read_csv("MDS_data_preprocessed/mirna.csv")
circrna = pl.read_csv("MDS_data_preprocessed/circrna.csv")
te_counts = pl.read_csv("MDS_data_preprocessed/te_counts.csv")

In [24]:
mrna["GENE_NAME"].to_list()[:10]

['RILPL1',
 'RAB4B',
 'TIGAR',
 'DNAH3',
 'RP11-432M8.3',
 'RPL23A',
 'ARL8B',
 'DACH1',
 'FMO2',
 'TTC30A']

In [ ]:
from src.data_preprocessing import CVSplitManager

CVSplitManager(
    "splits",
    n_splits=5,
    test_size=0.2,
    random_state=42,
).create_splits(df["1 disease"].to_numpy())

In [31]:
gtf = pl.read_csv("gencode.v47.basic.annotation.gtf",
    separator='\t',
    skip_rows=5,
    has_header=False,
)
gtf.columns = [
    'chromosome',      # chr1 in your example
    'source',         # HAVANA in your example
    'feature_type',   # gene in your example
    'start',          # 11121 in your example
    'end',           # 24894 in your example
    'score',         # . in your example
    'strand',        # + in your example
    'frame',         # . in your example
    'attributes'     # gene_id "ENSG00000290825.2"; etc. in your example
]

In [44]:
gtf.head()

chromosome,source,feature_type,start,end,score,strand,frame,attributes
str,str,str,i64,i64,str,str,str,str
"""chr1""","""HAVANA""","""gene""",11121,24894,""".""","""+""",""".""","""gene_id ""ENSG00000290825.2""; g…"
"""chr1""","""HAVANA""","""transcript""",11426,14409,""".""","""+""",""".""","""gene_id ""ENSG00000290825.2""; t…"
"""chr1""","""HAVANA""","""exon""",11426,11671,""".""","""+""",""".""","""gene_id ""ENSG00000290825.2""; t…"
"""chr1""","""HAVANA""","""exon""",12010,12227,""".""","""+""",""".""","""gene_id ""ENSG00000290825.2""; t…"
"""chr1""","""HAVANA""","""exon""",12613,12721,""".""","""+""",""".""","""gene_id ""ENSG00000290825.2""; t…"


In [58]:
attlist = gtf['attributes'].to_list()
# write as a csv into a file
with open('attributes.csv', 'w') as f:
    for item in attlist:
        f.write("%s\n" % item)

In [61]:
att_df = pl.read_csv("attributes.csv", separator=";", has_header=False, truncate_ragged_lines=True)

In [62]:
att_df

column_1,column_2,column_3,column_4,column_5,column_6
str,str,str,str,str,str
"""gene_id ""ENSG00000290825.2""""",""" gene_type ""lncRNA""""",""" gene_name ""DDX11L16""""",""" level 2""",""" tag ""overlaps_pseudogene""""",null
"""gene_id ""ENSG00000290825.2""""",""" transcript_id ""ENST0000083282…",""" gene_type ""lncRNA""""",""" gene_name ""DDX11L16""""",""" transcript_type ""lncRNA""""",""" transcript_name ""DDX11L16-264…"
"""gene_id ""ENSG00000290825.2""""",""" transcript_id ""ENST0000083282…",""" gene_type ""lncRNA""""",""" gene_name ""DDX11L16""""",""" transcript_type ""lncRNA""""",""" transcript_name ""DDX11L16-264…"
"""gene_id ""ENSG00000290825.2""""",""" transcript_id ""ENST0000083282…",""" gene_type ""lncRNA""""",""" gene_name ""DDX11L16""""",""" transcript_type ""lncRNA""""",""" transcript_name ""DDX11L16-264…"
"""gene_id ""ENSG00000290825.2""""",""" transcript_id ""ENST0000083282…",""" gene_type ""lncRNA""""",""" gene_name ""DDX11L16""""",""" transcript_type ""lncRNA""""",""" transcript_name ""DDX11L16-264…"
…,…,…,…,…,…
"""gene_id ""ENSG00000210195.2""""",""" transcript_id ""ENST0000038746…",""" gene_type ""Mt_tRNA""""",""" gene_name ""MT-TT""""",""" transcript_type ""Mt_tRNA""""",""" transcript_name ""MT-TT-201"""""
"""gene_id ""ENSG00000210195.2""""",""" transcript_id ""ENST0000038746…",""" gene_type ""Mt_tRNA""""",""" gene_name ""MT-TT""""",""" transcript_type ""Mt_tRNA""""",""" transcript_name ""MT-TT-201"""""
"""gene_id ""ENSG00000210196.2""""",""" gene_type ""Mt_tRNA""""",""" gene_name ""MT-TP""""",""" level 3""",""" hgnc_id ""HGNC:7494""""",null


In [65]:
genes = att_df.filter(pl.col("column_3").str.contains("gene_name"))

In [69]:
idg = genes.select(pl.col("column_1"), pl.col("column_3"))

In [80]:
# Cleaning column_1 and column_3
cleaned_df = idg.with_columns(
    pl.col("column_1").str.extract(r'gene_id "([^"]+)"', 1).alias("ensembl_id"),
    pl.col("column_3").str.extract(r'gene_name "([^"]+)"', 1).alias("gene_name")
).select(["ensembl_id", "gene_name"])

In [82]:
cleaned_df.write_csv("ensembl_ids_genes_map.csv")

In [84]:
gene_length_df.head(), cleaned_df.head()

(shape: (5, 5)
 ┌───────────────────┬──────┬────────┬─────────────────┬────────┐
 │ gene              ┆ mean ┆ median ┆ longest_isoform ┆ merged │
 │ ---               ┆ ---  ┆ ---    ┆ ---             ┆ ---    │
 │ str               ┆ i64  ┆ i64    ┆ i64             ┆ i64    │
 ╞═══════════════════╪══════╪════════╪═════════════════╪════════╡
 │ ENSG00000290825.2 ┆ 1762 ┆ 1762   ┆ 1762            ┆ 1762   │
 │ ENSG00000223972.6 ┆ 632  ┆ 632    ┆ 632             ┆ 632    │
 │ ENSG00000310526.1 ┆ 2004 ┆ 1876   ┆ 2843            ┆ 4211   │
 │ ENSG00000227232.6 ┆ 1380 ┆ 1380   ┆ 1380            ┆ 1380   │
 │ ENSG00000278267.1 ┆ 68   ┆ 68     ┆ 68              ┆ 68     │
 └───────────────────┴──────┴────────┴─────────────────┴────────┘,
 shape: (5, 2)
 ┌───────────────────┬───────────┐
 │ ensembl_id        ┆ gene_name │
 │ ---               ┆ ---       │
 │ str               ┆ str       │
 ╞═══════════════════╪═══════════╡
 │ ENSG00000290825.2 ┆ DDX11L16  │
 │ ENSG00000223972.6 ┆ DDX11L1   

In [83]:
gene_length_df = pl.read_csv("gene_length.txt", separator="\t")

In [92]:
gldf = gene_length_df.rename({"gene": "ensembl_id"})

In [93]:
gldf

ensembl_id,mean,median,longest_isoform,merged
str,i64,i64,i64,i64
"""ENSG00000290825.2""",1762,1762,1762,1762
"""ENSG00000223972.6""",632,632,632,632
"""ENSG00000310526.1""",2004,1876,2843,4211
"""ENSG00000227232.6""",1380,1380,1380,1380
"""ENSG00000278267.1""",68,68,68,68
…,…,…,…,…
"""ENSG00000292369.1""",773,773,773,773
"""ENSG00000292370.2""",1228,1228,1228,1228
"""ENSG00000292372.1""",1437,1437,1437,1437


In [96]:
gldf.join(cleaned_df, on="ensembl_id", how="inner").write_csv("gene_lengths.csv")

In [54]:
gtf['feature_type'].to_list()[:100]

['gene',
 'transcript',
 'exon',
 'exon',
 'exon',
 'exon',
 'gene',
 'transcript',
 'exon',
 'exon',
 'exon',
 'exon',
 'exon',
 'exon',
 'gene',
 'transcript',
 'exon',
 'exon',
 'exon',
 'exon',
 'exon',
 'exon',
 'exon',
 'exon',
 'exon',
 'transcript',
 'exon',
 'exon',
 'exon',
 'exon',
 'exon',
 'exon',
 'exon',
 'exon',
 'exon',
 'exon',
 'exon',
 'transcript',
 'exon',
 'exon',
 'exon',
 'exon',
 'exon',
 'exon',
 'exon',
 'exon',
 'exon',
 'exon',
 'exon',
 'exon',
 'exon',
 'transcript',
 'exon',
 'exon',
 'exon',
 'exon',
 'exon',
 'transcript',
 'exon',
 'exon',
 'exon',
 'exon',
 'exon',
 'exon',
 'exon',
 'exon',
 'exon',
 'transcript',
 'exon',
 'exon',
 'exon',
 'exon',
 'exon',
 'exon',
 'exon',
 'exon',
 'transcript',
 'exon',
 'exon',
 'exon',
 'exon',
 'exon',
 'exon',
 'exon',
 'exon',
 'exon',
 'exon',
 'exon',
 'transcript',
 'exon',
 'exon',
 'exon',
 'exon',
 'exon',
 'exon',
 'exon',
 'exon',
 'transcript',
 'exon',
 'exon']

In [50]:
gtf[5]['attributes'].to_list()[0].split(";")

['gene_id "ENSG00000290825.2"',
 ' transcript_id "ENST00000832828.1"',
 ' gene_type "lncRNA"',
 ' gene_name "DDX11L16"',
 ' transcript_type "lncRNA"',
 ' transcript_name "DDX11L16-264"',
 ' exon_number 4',
 ' exon_id "ENSE00004248703.1"',
 ' level 2',
 ' tag "basic"',
 ' tag "TAGENE"',
 '']